In [6]:
#from google.colab import auth
import os
import pandas as pd

In [7]:
path = "https://raw.githubusercontent.com/ekaratnida/Automated_Tools-/keep/Mini-bads3/cust_sensitivity.csv"
df = pd.read_csv(path)
df.head()

,CUST_CODE,BASKET_ID,PROD_CODE,STORE_CODE,price_sensitivity
0,CUST0000001098,2,8,1,1
1,CUST0000001392,1,17,1,0
2,CUST0000001437,1,2,1,0
3,CUST0000002218,1,8,1,0
4,CUST0000002678,1,2,1,0


In [8]:
x = df.iloc[:,1:-1]
print(x.shape)

(1860, 3)


In [9]:
y = df.iloc[:,-1:]
print(y.shape)

(1860, 1)


In [10]:
import statsmodels.api as sm
x = sm.add_constant(x)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
print(y_train.value_counts())
print(y_test.value_counts())

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


price_sensitivity
0                    1169
1                     319
dtype: int64
price_sensitivity
0                    292
1                     80
dtype: int64


In [11]:
logit_model   = sm.Logit(y_train,x_train)
train_result  = logit_model.fit()
train_result.summary2()

Optimization terminated successfully.
         Current function value: 0.515278
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit             Pseudo R-squared: 0.009    
Dependent Variable: price_sensitivity AIC:              1541.4676
Date:               2021-12-06 06:52  BIC:              1562.6883
No. Observations:   1488              Log-Likelihood:   -766.73  
Df Model:           3                 LL-Null:          -773.32  
Df Residuals:       1484              LLR p-value:      0.0042767
Converged:          1.0000            Scale:            1.0000   
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
const         -0.6575    0.2059  -3.1936  0.0014  -1.0610  -0.2540
BASKET_ID      0.0390    0.0586   0.6663  0.5052  -0.0758   0.1539
PROD_CODE     -0.0151    0.0079  -1.9088  0.0563  -0.0306   0.0004
STORE_CODE    -0.4739    0.1823  -2.5995  0.0093  -0.8312  -0.1166
=================================================================

"""

In [12]:
from sklearn.metrics import confusion_matrix

y_pred = train_result.predict(x_test)
#print(y_pred)
y_pred = (y_pred > 0.5).astype(int)
#print(y_pred)

confusion_matrix_train = confusion_matrix(y_test, y_pred)
print(confusion_matrix_train)

TruePositive = confusion_matrix_train[1,1]
print("TP=",TruePositive)

FalseNegative = confusion_matrix_train[1,0]
print("FN=",FalseNegative)

TrueNegative = confusion_matrix_train[0,0]
print("TN=",TrueNegative)

FalsePositive = confusion_matrix_train[0,1]
print("FP=",FalsePositive)


from sklearn.metrics import classification_report

target_names = ['not_sensitivity (0)', 'sensitivity (1)']
print(classification_report(y_test, y_pred, target_names=target_names))

[[292   0]
 [ 80   0]]
TP= 0
FN= 80
TN= 292
FP= 0
                     precision    recall  f1-score   support

not_sensitivity (0)       0.78      1.00      0.88       292
    sensitivity (1)       0.00      0.00      0.00        80

           accuracy                           0.78       372
          macro avg       0.39      0.50      0.44       372
       weighted avg       0.62      0.78      0.69       372



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Apply negative downsampling

In [13]:
y_train_pos = y_train[y_train['price_sensitivity']==1]
y_train_neg = y_train[y_train['price_sensitivity']==0]
#print("origin ", y_train_neg)
y_train_neg = y_train_neg.sample(int(y_train_neg.shape[0] * 0.30), random_state=42)
#print("down ", y_train_neg)
y_train_new = pd.concat([y_train_pos, y_train_neg])
x_train_new = x_train.loc[y_train_new.index]

print(y_train_new.value_counts())
print(y_test.value_counts())

price_sensitivity
0                    350
1                    319
dtype: int64
price_sensitivity
0                    292
1                     80
dtype: int64


<H1> Train model

In [14]:
choice = 2 #0 is statsmodel, 1 is sklearn, 2 is gridsearch
train_result = None

In [15]:
if choice == 0:
    logit_model   = sm.Logit(y_train_new, x_train_new)
    train_result  = logit_model.fit()

In [16]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import warnings
warnings.filterwarnings('ignore')

if choice == 1:
    model = LogisticRegression(fit_intercept = False, C = 1e9)
    train_result = model.fit(x_train_new, y_train_new)
    model.coef_


if choice == 2:
    from sklearn.model_selection import GridSearchCV
    grid={'C': [0.0001,0.001,0.01,0.1,1,10,1e2,1e3,1e6,1e9,1e12], "penalty":["l1","l2"], "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]}# l1 lasso l2 ridge
    logreg=LogisticRegression()
    train_result=GridSearchCV(logreg, grid, cv=7)
    train_result.fit(x_train_new,y_train_new)

    print("tuned hpyerparameters :(best parameters) ",train_result.best_params_)
    print("accuracy :",train_result.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
accuracy : 0.5576441102756893


In [17]:
y_pred = train_result.predict(x_test)
print(y_pred)

if choice == 0:
    y_pred = (y_pred > 0.5).astype(int)

confusion_matrix_train = confusion_matrix(y_test, y_pred)
print(confusion_matrix_train)

TruePositive = confusion_matrix_train[1,1]
print("TP=",TruePositive)

FalseNegative = confusion_matrix_train[1,0]
print("FN=",FalseNegative)

TrueNegative = confusion_matrix_train[0,0]
print("TN=",TrueNegative)

FalsePositive = confusion_matrix_train[0,1]
print("FP=",FalsePositive)

print(classification_report(y_test, y_pred, target_names=target_names))

[1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1
 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0
 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1
 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0
 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 1
 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1
 0 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0
 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1
 1 1]
[[142 150]
 [ 40  40]]
TP= 40
FN= 40
TN= 142
FP= 150
                     precision    recall  f1-score   support

not_sensitivity (0)       0.78      0.49      0.60       292
    sensitivity (1)       0.21      0.50      0.30        80

      